# Práctica 5 - Visión por Computador

### Autores

- Pablo Segura López
- Kilian Armas Pérez

## Tarea

In [1]:
import cv2
import time
import numpy as np
import os
os.chdir("./Material P5")
import FaceNormalizationUtils as faceutils
# My face detectors interface
import FaceDetectors
import math
from collections import deque
import random

In [ ]:
normalizatorHS = faceutils.Normalization()

# Face detectors interface
FDet = FaceDetectors.FaceDetector()

# Fonts
font = cv2.FONT_HERSHEY_SIMPLEX

# Webcam connection
cap = cv2.VideoCapture(0)
# Videos of ironman
closing_vid = cv2.VideoCapture("./IRONMAN/ironman-helmet-closing.gif")
opening_vid = cv2.VideoCapture("./IRONMAN/ironman-helmet-opening.gif")
# Get last frames
closing_vid.set(cv2.CAP_PROP_POS_FRAMES, int(closing_vid.get(cv2.CAP_PROP_FRAME_COUNT)) - 1)
opening_vid.set(cv2.CAP_PROP_POS_FRAMES, int(opening_vid.get(cv2.CAP_PROP_FRAME_COUNT)) - 1)
_, last_frame_closed = closing_vid.read()
_, last_frame_opened = opening_vid.read()
closing_vid.set(cv2.CAP_PROP_POS_FRAMES, 0)
opening_vid.set(cv2.CAP_PROP_POS_FRAMES, 0)

# Boolean variable to decide if the helmet is opening or closing and last time opened
opening = True
last_time_mouth_opened = time.time()

# Face detection and eye model setup (DNN + DLIB68)
imodoF = 2
imodoE = 1
# Stars in space background
n_rows_by_star = 6
stars = deque(maxlen=last_frame_opened.shape[0]//n_rows_by_star)
for i in range(last_frame_opened.shape[0]//n_rows_by_star):
    stars.append(random.randint(0,last_frame_opened.shape[1]))
# Position of the face in the image
helmet_x, helmet_y, helmet_w, helmet_h = 405, 130, 175, 175
# Green screen values
lower_green = np.array([35, 55, 55])
upper_green = np.array([90, 255, 255])
# Create the video file
frame_width = int(closing_vid.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(closing_vid.get(cv2.CAP_PROP_FRAME_HEIGHT))
output_video = cv2.VideoWriter('resultados.avi', cv2.VideoWriter_fourcc(*'XVID'), 20, (frame_width, frame_height))

while True:
    # Get frame
    t = time.time()
    ret, frame = cap.read()
    frame_ironman = last_frame_opened
    # Handle frames of ironman
    if opening:
        ret_open, frame_ironman = opening_vid.read()
        if not ret_open:
            frame_ironman = last_frame_opened
    else:
        ret_close, frame_ironman = closing_vid.read()
        if not ret_close:
            frame_ironman = last_frame_closed

    if ret:
        # For HS normalization
        B, G, R = cv2.split(frame)
        NormBGR = frame

        # Search face with a specific setup for face and eye detection
        values = FDet.SingleFaceEyesDetection(frame, FDet.FaceDetectors[imodoF], FDet.EyeDetectors[imodoE])
        if values is not None:
            face, eyes, shape = values

            #draws face container
            [x, y , w, h] = face
            if x > -1:
                cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)

                # draws eyes and mask if available
                [lex, ley, rex, rey] = eyes
                if lex > -1:
                    # Checks if mouth is open and acts in consequence 
                    x1,y1 = shape[62] # Central point of upper lip
                    x2,y2 = shape[66] # Central point of lower lip
                    x3,y3 = shape[63] # Next to central point of upper lip
                    distance_open = math.sqrt((x2 - x1)**2 + (y2 - y1)**2)
                    distance_neighbor = math.sqrt((x3 - x1)**2 + (y3 - y1)**2)
                    current_time_mouth_opened = time.time()
                    # Starts opening or closing the helmet if the mouth has opened
                    if distance_open > distance_neighbor*2 and current_time_mouth_opened - last_time_mouth_opened >= 1.5:
                        last_time_mouth_opened = current_time_mouth_opened
                        opening = not opening
                        closing_vid.set(cv2.CAP_PROP_POS_FRAMES, 0)
                        opening_vid.set(cv2.CAP_PROP_POS_FRAMES, 0)

                    # Normalize and show
                    # color channels
                    normalizatorHS.normalize_gray_img(B, lex, ley, rex, rey, faceutils.Kind_wraping.FACE)
                    Bnorm = normalizatorHS.normf_image
                    normalizatorHS.normalize_gray_img(G, lex, ley, rex, rey, faceutils.Kind_wraping.FACE)
                    Gnorm = normalizatorHS.normf_image
                    normalizatorHS.normalize_gray_img(R, lex, ley, rex, rey, faceutils.Kind_wraping.FACE)
                    Rnorm = normalizatorHS.normf_image
                    NormBGR = cv2.merge((Bnorm, Gnorm, Rnorm))
        # Create background
        background = frame_ironman*0
        # Create new star for first row
        stars.append(random.randint(0, frame_ironman.shape[1]-1))
        # Represent stars
        for i, star in enumerate(stars):
            cv2.circle(background, (star,i*n_rows_by_star), 2, (255,255,255), -1)
        # Put face in place
        background[helmet_y:helmet_y + helmet_h, helmet_x:helmet_x + helmet_w] = cv2.resize(NormBGR, (helmet_w, helmet_h))
        # Put ironman armor in place
        mask = cv2.inRange(cv2.cvtColor(frame_ironman, cv2.COLOR_BGR2HSV), lower_green, upper_green)
        mask_inv = cv2.bitwise_not(mask)
        background = cv2.bitwise_and(background, background, mask=mask)
        foreground = cv2.bitwise_and(frame_ironman, frame_ironman, mask=mask_inv)
        result = cv2.add(background, foreground)
        # Show resulting image
        cv2.putText(result, "OPEN YOUR MOUTH TO OPEN/CLOSE IRONMAN ARMOR'S HELMET", (10, 20), font, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
        cv2.imshow('IRONMAN flying through space', result)
        output_video.write(result)
        # Esc to finish
        tec = cv2.waitKey(40)
        if tec & tec == 27:  # Esc
            break

# Close windoews and release camera
cap.release()
cv2.destroyAllWindows()